In [1]:
import pandas as pd
import os

print("開始進行資料合併 (Data Merge)")

# --- 設定路徑  ---
# 原始資料來源
raw_path = '../data/raw'
# 處理後資料去處
processed_path = '../data/processed'

# 確保輸出資料夾存在
os.makedirs(processed_path, exist_ok=True)

# 1. 讀取資料 (從 raw 讀取)
print(" 讀取原始 CSV")
try:
    df_status = pd.read_csv(f'{raw_path}/station_status.csv')
    df_info = pd.read_csv(f'{raw_path}/station_info.csv')
    # 確保你已經跑過 02_weather_etl.ipynb 產生這份檔案
    df_weather = pd.read_csv(f'{raw_path}/taipei_weather_hourly.csv') 
    print(" 原始資料讀取成功")
except FileNotFoundError as e:
    print(f" 錯誤：找不到檔案，詳細資訊：{e}")
    raise

# 2. 合併 YouBike 資料
print(" 合併 YouBike (Status + Info)...")
df_status['station_no'] = df_status['station_no'].astype(str)
df_info['station_no'] = df_info['station_no'].astype(str)
df_youbike = pd.merge(df_status, df_info, on='station_no', how='left')

# 3. 處理時間格式以進行合併
df_youbike['record_time'] = pd.to_datetime(df_youbike['record_time'])
df_youbike['hour_key'] = df_youbike['record_time'].dt.floor('h') # 取到小時

df_weather['record_time'] = pd.to_datetime(df_weather['record_time'])
df_weather = df_weather.rename(columns={'record_time': 'hour_key'})

# 4. 合併天氣資料
print(" 合併天氣資料")
df_final = pd.merge(df_youbike, df_weather, on='hour_key', how='left')

# 5. 存檔到 processed
output_file = f'{processed_path}/youbike_weather_merged.csv'
df_final.to_csv(output_file, index=False)

print(f"\n 最終資料已產生，位置在：{output_file}")

開始進行資料合併 (Data Merge)
 讀取原始 CSV
 原始資料讀取成功
 合併 YouBike (Status + Info)...
 合併天氣資料

 最終資料已產生，位置在：../data/processed/youbike_weather_merged.csv
